In [40]:
import os
os.chdir("/home/vic3/github/TradeSmartPlus/TradeSmart_Plus")
print(os.getcwd())

/home/vic3/github/TradeSmartPlus/TradeSmart_Plus


In [41]:
import sys
sys.path.append("/home/vic3/github/TradeSmartPlus/TradeSmart_Plus/scripts")

In [42]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [56]:
! pip install -r requirements.txt


In [57]:
# TradeSmart_Plus.ipynb

# ## 1. Import necessary libraries and scripts

import os
import pandas as pd
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from scripts.fetch_stock_data import fetch_stock_data
from scripts.models import train_random_forest, train_xgboost, train_lstm_model
from scripts.backtest import backtest_strategy
from scripts.generate_report import generate_report
from scripts.sentiment_analysis import get_sentiment

In [58]:
import yfinance as yf
import talib as ta
import pandas as pd

In [59]:
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from sklearn.metrics import accuracy_score

In [60]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

In [61]:
from fpdf import FPDF

In [62]:
import gym
from gym import spaces
import numpy as np

In [63]:
import unittest
from scripts.backtest_strategy import backtest_strategy
import pandas as pd

In [51]:
# ## 2. Set up the stock symbols and parameters

symbols = ['AAPL', 'MSFT', 'GOOGL', 'SPY']
start_date = "2018-01-01"
initial_balance = 10000

# ## 3. Fetch stock data and perform feature engineering

stock_data = fetch_stock_data(symbols, start_date=start_date)

# Display stock data to verify
print(stock_data['AAPL'].head())

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


                 Open       High        Low      Close  Adj Close     Volume  \
Date                                                                           
2018-01-02  42.540001  43.075001  42.314999  43.064999  40.568932  102223600   
2018-01-03  43.132500  43.637501  42.990002  43.057499  40.561855  118071600   
2018-01-04  43.134998  43.367500  43.020000  43.257500  40.750267   89738400   
2018-01-05  43.360001  43.842499  43.262501  43.750000  41.214218   94640000   
2018-01-08  43.587502  43.902500  43.482498  43.587502  41.061146   82271200   

            SMA_20  RSI  EMA_20  
Date                             
2018-01-02     NaN  NaN     NaN  
2018-01-03     NaN  NaN     NaN  
2018-01-04     NaN  NaN     NaN  
2018-01-05     NaN  NaN     NaN  
2018-01-08     NaN  NaN     NaN  


In [52]:
# ## 4. Sentiment Analysis (Optional - Using VADER)

# Example of analyzing sentiment from a news article
news_article = "The company's stock surged after reporting record earnings."
sentiment_result = get_sentiment(news_article)
print(f"Sentiment Analysis Result: {sentiment_result}")

# ## 5. Prepare Data for Model Training

Sentiment Analysis Result: 0.0


In [53]:
def prepare_data(ticker_data):
    # Calculate technical indicators
    ticker_data['SMA_20'] = ticker_data['Close'].rolling(window=20).mean()
    ticker_data['RSI'] = ta.RSI(ticker_data['Close'], timeperiod=14)
    
    # Create a DataFrame for features and drop rows where any of the features are NaN
    X = ticker_data[['SMA_20', 'RSI']].dropna()
    
    # Define target 'y' for binary classification (1 if the next day price is higher, else 0)
    y = np.where(ticker_data['Close'].shift(-1) > ticker_data['Close'], 1, 0)
    
    # Convert y to a pandas Series with the same index as X
    y = pd.Series(y, index=ticker_data.index)
    
    # Align 'y' with the same index as the feature DataFrame 'X'
    y = y.loc[X.index]
    
    # Split data into training, validation, and testing sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25, random_state=42)
    
    return X_train, X_val, X_test, y_train, y_val, y_test

In [54]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from sklearn.metrics import accuracy_score, classification_report
import talib as ta


In [67]:
# 1. Import Libraries and Custom Functions
from backtest import generate_signals

symbols = ['AAPL']
stock_data = fetch_stock_data(symbols)

all_backtests = {}

# 2. Loop Through Tickers
for ticker in symbols:
    ticker_data = stock_data[ticker]
    
    # Prepare data
    X_train, X_val, X_test, y_train, y_val, y_test = prepare_data(ticker_data)
    
    # Train models
    rf_model, rf_accuracy = train_random_forest(X_train, y_train, X_val, y_val)
    xgb_model, xgb_accuracy = train_xgboost(X_train, y_train, X_val, y_val)
    lstm_model, lstm_accuracy = train_lstm_model(X_train, y_train, X_val, y_val)
    
    # Generate signals
    rf_signals = generate_signals(ticker_data)
    xgb_signals = generate_signals(ticker_data)
    lstm_signals = generate_signals(ticker_data)

    # Backtest
    rf_backtest = backtest_strategy(ticker_data, rf_signals)
    xgb_backtest = backtest_strategy(ticker_data, xgb_signals)
    lstm_backtest = backtest_strategy(ticker_data, lstm_signals)
    
    all_backtests[ticker] = {
        'RandomForest': rf_backtest,
        'XGBoost': xgb_backtest,
        'LSTM': lstm_backtest
    }
    
    print(f"{ticker}: RF Accuracy: {rf_accuracy}, XGB Accuracy: {xgb_accuracy}, LSTM Accuracy: {lstm_accuracy}")


[*********************100%***********************]  1 of 1 completed
/home/vic3/anaconda3/lib/python3.11/site-packages/xgboost/core.py:158: UserWarning: [12:03:01] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
2024-10-17 12:03:15.935817: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2024-10-17 12:03:15.948170: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtyp

Epoch 1/20


2024-10-17 12:03:17.888411: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2024-10-17 12:03:17.894318: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2024-10-17 12:03:17.898290: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

30/32 [===========================>..] - ETA: 0s - loss: 0.6948

2024-10-17 12:03:26.426933: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2024-10-17 12:03:26.432577: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2024-10-17 12:03:26.435428: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

32/32 [==============================] - 10s 53ms/step - loss: 0.6949 - val_loss: 0.6908
Epoch 2/20
32/32 [==============================] - 0s 11ms/step - loss: 0.6961 - val_loss: 0.6851
Epoch 3/20
32/32 [==============================] - 0s 11ms/step - loss: 0.6946 - val_loss: 0.6881
Epoch 4/20
32/32 [==============================] - 0s 13ms/step - loss: 0.6920 - val_loss: 0.6853
Epoch 5/20
32/32 [==============================] - 0s 11ms/step - loss: 0.6930 - val_loss: 0.6874
Epoch 6/20
32/32 [==============================] - 0s 9ms/step - loss: 0.6942 - val_loss: 0.6880
Epoch 7/20
32/32 [==============================] - 0s 10ms/step - loss: 0.6932 - val_loss: 0.6873
Epoch 8/20
32/32 [==============================] - 0s 9ms/step - loss: 0.6923 - val_loss: 0.6871
Epoch 9/20
32/32 [==============================] - 0s 9ms/step - loss: 0.6920 - val_loss: 0.6885
Epoch 10/20
32/32 [==============================] - 0s 9ms/step - loss: 0.6933 - val_loss: 0.6853
Epoch 11/20
32/32 [====

2024-10-17 12:03:33.155955: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2024-10-17 12:03:33.159382: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2024-10-17 12:03:33.161045: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

11/11 [==============================] - 1s 4ms/step


KeyError: 0